In [1]:
import numpy as np
import numpy.matlib
from scipy.fft import fft, ifft, fft2, ifft2
from scipy import integrate
# import matplotlib.pyplot as plt
# import matplotlib.animation as animation

# import time
# import sys
# numpy.set_printoptions(threshold=sys.maxsize)

# Problem 1 

In [2]:
m = 3
alpha = 0
n = 64

dt = 0.5
T = 25
tspan = np.arange(0,25.5,dt)

L = 20 
x2 = np.linspace(-L/2,L/2,n+1) 
x=x2[0:-1] 
y=x.copy()

B = 1 
D1 = 0.1
D2 = D1 

#Scale our k values to the length-L domain
r1=np.arange(0,n/2,1)
r2=np.arange(-n/2,0,1)
kx = (2*np.pi/L)*np.concatenate((r1,r2))
ky = kx.copy()

In [3]:
#Do the same for the k values
[KX,KY]=np.meshgrid(kx,ky)

#This term shows up in the Laplacian
K = KX**2+KY**2
Kvec = K.reshape((n**2), order = 'F')

### 1a & 1b -  A1 and A2

In [4]:
X, Y = np.meshgrid(x,y)
m = 3
alpha = 0
u = (np.tanh(np.sqrt(X**2+Y**2))-alpha)*np.cos(m*np.angle(X+1j*Y) - np.sqrt(X**2+Y**2))
v = (np.tanh(np.sqrt(X**2+Y**2))-alpha)*np.sin(m*np.angle(X+1j*Y) - np.sqrt(X**2+Y**2))

In [5]:
A1 = X.copy()

In [6]:
A2 = u.copy()

### 1c - A3

In [7]:
u0_hat = fft2(u)
# u0hat_vector = u0_hat.T.reshape((n**2), order = 'F')
# u0hat_vector = u0_hat.T.reshape((n**2))
u0hat_vector = u0_hat.reshape((n**2), order = 'F')

v0_hat = fft2(v)
# v0hat_vector = v0_hat.T.reshape((n**2), order = 'F')
# v0hat_vector = v0_hat.T.reshape((n**2))
v0hat_vector = v0_hat.reshape((n**2), order = 'F')

In [8]:
# u0_hat.shape

In [9]:
A3 = np.real(fft2(u))

### 1d - A4

In [10]:
y0hat = np.concatenate((u0hat_vector, v0hat_vector), axis=0)

In [11]:
# y0hat = np.hstack((u0hat_vector, v0hat_vector))

In [12]:
A4 = np.imag(y0hat)

In [13]:
A4 = A4.reshape(8192,1)

### 1e - A5

In [14]:
#reaction-diffusion system ODE (take the real part at the end?)
def FFT_RD(t, yhat, n):
    
    #Unstack and Convert back to matrix
    u_f = yhat[:4096].reshape(n,n,order='F') #double check this
    v_f = yhat[4096:].reshape(n,n,order='F') #double check this

    #Convert back to physical space 
    u = np.real(ifft2(u_f))
    v = np.real(ifft2(v_f))

    #defining lambdaA and wA  
    lambda_A = 1 - (u**2 + v**2) #nxn matrix
    w_A = -B*(u**2 + v**2) #nxn matrix
    
    #Convert everything back and return a vector in the Fourier space
    dudt = fft2(lambda_A*u - w_A*v).reshape(n**2,order='F') + (-D1 * Kvec * yhat[:4096]) 
    dvdt = fft2(w_A*u - lambda_A*v).reshape(n**2,order='F') + (-D2 * Kvec * yhat[4096:])
    
    #stack dudt and dvdt 
    rhs = np.concatenate((dudt, dvdt), axis=0)
    
    return rhs

In [15]:
sol = integrate.solve_ivp(lambda t, yhat: FFT_RD(t, yhat, n), [0, T], y0hat, t_eval = tspan)

In [16]:
A5 = np.real(sol.y)

### 1e - A6

In [17]:
A6 = np.imag(sol.y)

### 1f - A7

In [18]:
A7 = sol.y[:4096,4].reshape(4096,1)

### 1g - A8

In [19]:
A8 = A7.reshape(64, 64, order = 'F')

### 1h - A9

In [20]:
A9 = np.real(ifft2(A8))
# y = ifft(sol.y, n=n,axis=0)
#We will plot the modulus squared, which is real
# y = abs(y.T)**2

In [21]:
# A9.shape

In [22]:
# Animation plotting

In [23]:
# wfvecsol = sol.y

In [24]:
# sol.y[:25][:64].reshape(n,n)

In [25]:
# #return to solution space and show as movie
# fig = plt.figure()
# ax = plt.axes(xlim=(np.min(x), np.max(x)), ylim=(np.min(y), np.max(y)), xlabel='x', ylabel='y')

# # animation function
# def animate(i):
#     global cont
#     if i in range(0,52):
#         curw=np.real(ifft2(wfvecsol[:,i][:n**2].reshape(n,n, order = 'F')))
#     else:
#         pass
#     cont = plt.pcolor(X, Y, curw)
#     return cont

# anim = animation.FuncAnimation(fig, animate, frames=81, repeat=False)
# anim.save('animation.mp4', writer=animation.FFMpegWriter())

### Problem 2 

In [26]:
# from cheb import *

In [27]:
def cheb(N):
    # N is the number of points in the interior.
    if N==0:
        D = 0
        x = 1
        return D, x
    vals = np.linspace(0, N, N+1)
    x = np.cos(np.pi*vals/N)
    x = x.reshape(-1, 1)
    c = np.ones(N-1)
    c = np.pad(c, (1,), constant_values = 2)
    c *= (-1)**vals
    c = c.reshape(-1, 1)
    X = np.tile(x, (1, N+1))
    dX = X-X.T                  
    D  = (c*(1/c.T))/(dX+(np.eye(N+1)))       #off-diagonal entries
    D  = D - np.diag(sum(D.T))                #diagonal entries

    return D, x

In [28]:
N = 30 
L = 20 

[D, x] = cheb(N)
x = x.reshape(N+1)

# Derivative operator squared is second derivative
DD2 = D@D

# Rescale x 
x = x*L/2

#This rescales the derivative 
DD2 = 4/(L**2)*DD2

# Apply BCs
DD2 = DD2[1:-1,1:-1]
x2 = x[1:-1]
y2 = x2.copy()

[X,Y] = np.meshgrid(x2,y2)
m = 2
alpha = 1

#Initial conditions
u = (np.tanh(np.sqrt(X**2+Y**2))-alpha)*np.cos(m*np.angle(X+1j*Y) - np.sqrt(X**2+Y**2))
v = (np.tanh(np.sqrt(X**2+Y**2))-alpha)*np.sin(m*np.angle(X+1j*Y) - np.sqrt(X**2+Y**2))

In [29]:
u0_vec = u.T.reshape((N-1)**2)
v0_vec = v.T.reshape((N-1)**2)

### 2a - A10

In [30]:
I = np.eye(len(DD2))

Lap = np.kron(DD2, I) + np.kron(I,DD2)

In [31]:
A10 = Lap.copy()

### 2b - A11 

In [32]:
A11 = Y.copy()

### 2c - A12

In [33]:
A12 = v.copy()

### 2d - A13

In [34]:
y0vec = np.hstack([u0_vec, v0_vec])

In [35]:
A13 = y0vec.reshape(1682,1)

### 2e - A14

In [36]:
#reaction-diffusion system ODE
def Chebyshev_RD(t, y, Lap):
    
    #define u and v from y 
    u = y[:(N-1)**2]
    v = y[(N-1)**2:]
        
    #defining lambdaA and wA  
    lambda_A = 1 - (u**2 + v**2)
    w_A = -B*(u**2 + v**2)
    
    dudt = lambda_A*u - w_A*v + D1*(Lap@u) 
    dvdt = w_A*u - lambda_A*v + D2*(Lap@v)
    
    #reshape dudt and dvdt
    dudt = dudt.reshape((N-1)**2,)
    dvdt = dvdt.reshape((N-1)**2,)
    
    #stack dudt and dvdt 
    rhs = np.hstack([dudt, dvdt])
    
    return rhs

In [37]:
# tic_gaus = time.time()
sol_cheb = integrate.solve_ivp(lambda t, y: Chebyshev_RD(t, y, Lap), [0, T], y0vec, t_eval = tspan)
# toc_gaus = time.time()
# print(toc_gaus-tic_gaus)

In [38]:
A14 = sol_cheb.y.T

### 2f - A15

In [39]:
A15 = sol_cheb.y[(N-1)**2:,4].reshape((N-1)**2,1)

### 2g - A16

In [40]:
A15_reshape = A15.reshape(N-1, N-1).T

In [41]:
# A15_reshape.shape

In [42]:
zero = np.zeros(N-1)

In [43]:
A16 = np.vstack([zero,A15_reshape,zero])

In [44]:
zero_cols = np.zeros(N+1).reshape(31,1)

In [45]:
A16 = np.hstack([zero_cols,A16,zero_cols])